In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt
import os

In [ ]:
def cargar_imagen(ruta_imagen):
    """
    Carga una imagen desde la ruta dada en BGR y la convierte a RGB para 
    visualizar con Matplotlib, además de la escala de grises.
    Retorna:
        - img_rgb: Imagen en formato RGB
        - img_gray: Imagen original en escala de grises
    """
    img_bgr = cv2.imread(ruta_imagen)
    if img_bgr is None:
        raise ValueError(f"No se pudo cargar la imagen en la ruta: {ruta_imagen}")

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    return img_rgb, img_gray

In [ ]:
def mostrar_imagen(img_rgb, titulo="Imagen"):
    """
    Muestra una imagen en RGB con Matplotlib.
    """
    plt.imshow(img_rgb)
    plt.title(titulo)
    plt.axis("off")
    plt.show()

In [ ]:
def wavelet_alta_frecuancia(img_gray, wavelet='db1'):
    """
    Aplica la Transformada Wavelet Discreta 2D a la imagen y devuelve los coeficientes de alta frecuencia.
    """
    coeficientes = pywt.wavedec2(img_gray, wavelet=wavelet, level=1) #wavedec2 descompone la imagen en coeficientes de alta frecuencia; level indica qeu solo es una descomposición
    coef_alta_frecuencia = coeficientes[1:]  # Ignoramos el coeficiente de aproximación, ya que este devuelve baja frecuencia y no nos interesa
    return coef_alta_frecuencia # Devuelve coeficiente diagonal, horizontal, diagonal 
#wavelet_alta_frecuancia(cargar_imagen("000004_blur.png")[1], wavelet='db1')

In [ ]:

def valor_nitidez(coef_alta_frecuencia):
    """
    Calcula un valor de nitidez basandose en los coeficientes diagonal, horizontal y diagonal.
    """
    total_sum = 0
    total_count = 0 

    for coef in coef_alta_frecuencia:   # Recorro los tres coeficientes 
        for i in coef:    
            for j in i:                 # Recorro todos los valores del coeficiente
                total_sum += abs(j)     # Sumo los valores de los coeficientes
                total_count += 1        # Cuento el nº de coef

    if total_count == 0:
        return 0  

    return total_sum / total_count      # Devuelvo el valor medio de los coeficientes (si devuelvo el total no podrá elegir bien si las iamgenes son de distinto tamaño)


valor_nitidez(wavelet_alta_frecuancia(cargar_imagen("000004_blur.png")[1], wavelet='db1'))

In [ ]:

def compute_sharpness_metric(coeffs_HF):
    """
    Calcula una métrica de nitidez basada en la energía de los coeficientes de alta frecuencia.
    """
    energy = sum(np.sum(np.abs(detail_coeffs)) for detail_coeffs in coeffs_HF)
    return energy
compute_sharpness_metric(wavelet_alta_frecuancia(cargar_imagen("000004_blur.png")[1], wavelet='db1'))

In [ ]:

def compute_sharpness_metric2(coeffs_HF):
    """
    Calcula una métrica de nitidez basada en la media de los coeficientes de alta frecuencia.
    """
    # Concatenar todos los coeficientes en una lista
    all_coeffs = np.concatenate([coeff.flatten() for coeff in coeffs_HF])
    
    # Calcular la media de los coeficientes
    mean_value = np.mean(np.abs(all_coeffs))  # Tomamos el valor absoluto y luego calculamos la media
    return mean_value
#compute_sharpness_metric2(wavelet_alta_frecuancia(cargar_imagen("000004_blur.png")[1], wavelet='db1'))

In [ ]:
def plot_wavelet_coeffs(coeffs_HF):
    """
    Visualiza los coeficientes de alta frecuencia de la Transformada Wavelet.
    """
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    titles = ['Horizontal Detail', 'Vertical Detail', 'Diagonal Detail']
    
    for i, (ax, coeff) in enumerate(zip(axes, coeffs_HF[0])):  
        ax.imshow(np.abs(coeff), cmap='gray')
        ax.set_title(titles[i])
        ax.axis('off')

    plt.show()


plot_wavelet_coeffs(wavelet_alta_frecuancia(cargar_imagen("000004_blur.png")[1], wavelet='db1'))

In [ ]:
def procesar_imagenes_carpeta_wavelet(
    carpeta_entrada="images",
    carpeta_salida="images_high_frec",
    wavelet='db1' #Tipo de wavelet
):
    """
    Recorre todas las subcarpetas dentro de 'carpeta_entrada'.
    - Para cada archivo .png, aplica la transformada de wavelet
    - Guarda la imagen resultante en 'carpeta_salida', manteniendo la misma estructura.
    - Genera un archivo .txt con el valor de nitidez de cada imagen procesada.

    Parámetros:
    - carpeta_entrada: ruta de la carpeta de entrada.
    - carpeta_salida: ruta de la carpeta donde se guardarán los resultados.
    """
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'wavelet'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "wavelet")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar las varianzas
        ruta_txt = os.path.join(carpeta_salida_sub, "info_nitidez_wavelet.txt")
        
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Coeficientes de alta frecuencia (Transformada de wavelet)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Valores mayores suelen corresponder a más bordes y mayor nitidez.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(".png"):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en escala de grises
                    img_bgr = cv2.imread(ruta_imagen_entrada)
                    if img_bgr is None:
                        continue
                    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
                    
                    # Obtengo coeficientes de alta frecuencia usando transformada de wavelet 
                    coeficientes_alta_frecuencia = wavelet_alta_frecuancia(img_gray, wavelet=wavelet)
                    
                    # Calcular valor medio de los coeficientes 
                    coef_medio = compute_sharpness_metric(coeficientes_alta_frecuencia) 
                    ##############################
                    ###CAmbiar por valor nitide###
                    ##############################



                    # Guardar imágenes de los coeficientes de alta frecuencia
                    nombre_salida = os.path.splitext(filename)[0]  # Usar el nombre de archivo base para los coeficientes
                    
                    # Guardar cada coeficiente como una imagen
                    titles = ['Horizontal Detail', 'Vertical Detail', 'Diagonal Detail']
                    for i, coeff in enumerate(coeficientes_alta_frecuencia[0]):
                        # Crear una nueva imagen para cada coeficiente
                        coef_frec_escalada = np.abs(coeff).astype(np.uint8)
                        
                        # Construir ruta de salida para cada coeficiente
                        ruta_imagen_salida = os.path.join(carpeta_salida_sub, f"{nombre_salida}_coef_{titles[i].replace(' ', '_')}.png")
                        
                        # Guardar imagen del coeficiente
                        cv2.imwrite(ruta_imagen_salida, coef_frec_escalada)




                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> Coeficiente medio: {coef_medio:.3f}\n")

    print("Procesamiento completado con transformada de wavelet.")

In [ ]:
# Ejecuta el procesamiento
procesar_imagenes_carpeta_wavelet(
    carpeta_entrada="images",
    carpeta_salida="images_high_frec",

)